In [1]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import os
import numpy as np
os.chdir('../..')

from src.utils import *

In [2]:
speeds_df = pd.read_csv(resources_path('dataset', 'originals', 'speeds_train.csv.gz'))
sensors_df = pd.read_csv(resources_path('dataset', 'originals', 'sensors.csv.gz'))

In [4]:
%time speeds_df = speeds_df[speeds_df[KEY] == 0]

CPU times: user 7.84 ms, sys: 32.5 ms, total: 40.3 ms
Wall time: 38.9 ms


## Creating skeleton dataframe

In [98]:
print(sensors_df.shape)
sensors_df.sort_values([KEY, KM]).head(3)

(2269, 5)


,KEY,EMERGENCY_LANE,KM,LANES,ROAD_TYPE
932,0.0,1,333,2,1
1642,0.0,1,337,2,1
472,0.0,1,342,2,1


In [ ]:
min_time = pd.to_datetime(speeds_df.DATETIME_UTC).astype('int').min()
max_time = pd.to_datetime(speeds_df.DATETIME_UTC).astype('int').max()
min_time = min_time//(10**9)
max_time = max_time//(10**9)
datetimes = np.arange(min_time, max_time, 15*60)
datetimes = datetimes*(10**9)

In [ ]:
datetimes = pd.to_datetime(datetimes)
datetimes = pd.DataFrame({DATETIME: datetimes})
datetimes.shape

In [ ]:
datetimes['key'] = 0
sensors = sensors_df[[KEY, KM]].drop_duplicates()
print(sensors.shape)
sensors['key'] = 0
skeleton_train_df = pd.merge(sensors, datetimes, on='key')[[KEY, KM, DATETIME]].sort_values([KEY, KM, DATETIME])
skeleton_train_df.head(10)

In [ ]:
# skeleton_train_df.to_csv(resources_path('dataset', 'preprocessed', 'skeleton_train.csv.gz'))

## Fill gaps for the first sensor

In [3]:
skeleton_train_df = pd.read_csv(resources_path('dataset', 'preprocessed', 'skeleton_train.csv.gz'))

In [174]:
speeds = speeds_df# [(speeds_df.KEY == 0) & ((speeds_df.KM == 333) | (speeds_df.KM == 333))]
skeleton = skeleton_train_df[[KEY, KM, DATETIME]]#[(skeleton_train_df.KEY == 0) & ((skeleton_train_df.KM == 333) | (skeleton_train_df.KM == 333))]
complete_df = pd.merge(skeleton, speeds, on=[KEY, KM, DATETIME], how='left')[[KEY, KM, DATETIME, SPEED_AVG, SPEED_SD, SPEED_MIN, SPEED_MAX, N_CARS]]
complete_df.sort_values(DATETIME).head(2)

,KEY,KM,DATETIME_UTC,SPEED_AVG,SPEED_SD,SPEED_MIN,SPEED_MAX,N_VEHICLES
0,0.0,333,2018-09-01 00:00:00,231.125907,34.573767,169.89,440.096,150.0
759945,14.0,1049,2018-09-01 00:00:00,NaN,NaN,NaN,NaN,NaN


In [175]:
complete_df.isnull().any(axis=1).sum()

8132259

In [176]:
complete_df['IMPUTED'] = complete_df.isnull().any(axis=1)
complete_df[complete_df['IMPUTED'] == True].sort_values([DATETIME]).head(2)

,KEY,KM,DATETIME_UTC,SPEED_AVG,SPEED_SD,SPEED_MIN,SPEED_MAX,N_VEHICLES,IMPUTED
4315090,87.0,1496,2018-09-01 00:00:00,NaN,NaN,NaN,NaN,NaN,True
2314775,47.0,1782,2018-09-01 00:00:00,NaN,NaN,NaN,NaN,NaN,True


In [177]:
complete_df[DATETIME] = pd.to_datetime(complete_df[DATETIME])
complete_df = complete_df.set_index([DATETIME])
complete_df.head(2)

,KEY,KM,SPEED_AVG,SPEED_SD,SPEED_MIN,SPEED_MAX,N_VEHICLES,IMPUTED
DATETIME_UTC,,,,,,,,
2018-09-01 00:00:00,0.0,333,231.125907,34.573767,169.890,440.096,150.0,False
2018-09-01 00:15:00,0.0,333,225.915947,41.437726,98.698,436.860,225.0,False


In [178]:
complete_df = complete_df.interpolate(method='time', limit=3, limit_area='inside')
complete_df.isnull().any(axis=1).sum()

7310755

In [179]:
complete_df.to_csv(resources_path('dataset', 'preprocessed', 'speeds_train_imputed_time.csv.gz'))